# Extraction des tweets: Covid-19 vaccine

## Importer les packages

- **Tweepy** : Le module Tweepy nous permet d’interroger de façon très simple avec l’API Twitter afin de récupérer les Tweets.
- **OAuth** (Open Authentification) : est un protocole libre de gestion des autorisations sur une API. Elle permet d'utiliser de façon sécurisée une API. **OAuthHandler** pour s'authentifier sur **Twitter**.
- **NumPy** : est une bibliothèque pour le langage de programmation Python, destinée à manipuler des matrices ou tableaux multidimensionnels ainsi que des fonctions mathématiques opérant sur ces tableaux.
- **Pandas** : est une bibliothèque écrite pour le langage de programmation Python permettant la manipulation et l'analyse des données. 


In [1]:
import tweepy
from tweepy import API 
from tweepy import Cursor
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler 
from tweepy import Stream
import numpy as np
import pandas as pd

In [2]:
consumer_key="lCmUK0wkuQaPAsATiWaeFyx9I"
consumer_secret="RH9qCxUDOyzGaP9Sak0lX02t2vHgpzovslgwonNEuxr2C2CLaB"
access_token="1477255548294897665-tLSZnzoD5oRCSFxvL2xHGbGQ7YNloF"
access_token_secret="z5mghK3BXUCY7gij39eeNaobD5tZOj5w9AMuQrrtZRAlA"

## Authentification Twitter

Passons maintenant à lier un compte à une application Twitter. C'est le cœur du process **OAuth** : Le principe est de permettre à une application donnée de se connecter à une API comme si elle était l'utilisateur lié.

- L’application se connecte à l'API pour demander un unauthorized_access_token en fournissant sa consumer_key et sa consumer_secret
- L’API renvoie un unauthorized_access_token à l'application
- L’application redirige l'utilisateur vers l'API en lui fournissant en paramètre l'unauthorized_access_token et une url de callback (l'utilisateur ne manipule donc jamais la consumer_key et la consumer_secret)
- L’utilisateur, sur la page de l'API, va donc autoriser l'accès
- L’API renvoie l'utilisateur sur l'url de callback donnée en paramètre et fournissant un access_token et un secret_access_token


Le concept de **Oauth** : fournir un moyen de connexion sécurisé à la fois pour l'application et pour l'utilisateur de l'application

In [3]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

## Chercher les tweets avec le mot « covid19 vaccine »

La sélection des tweets à analyser est réalisée au moyen de filtres basés sur le mot (**covid19 vaccine**). Ce mot permet d’identifier précisément la thématique supposée du tweet. 

In [4]:
for tweet in api.search('covid19 vaccine'):
    print(tweet)

Status(_api=<tweepy.api.API object at 0x000001AC464E31F0>, _json={'created_at': 'Tue Jan 11 20:10:17 +0000 2022', 'id': 1480995500056846337, 'id_str': '1480995500056846337', 'text': 'RT @redstarneil: Update on pop up #COVID19 vaccination this week\n #VaccinesWork\nLocations include \nTue 11th @KensalRLibrary\nWed 12th #Kilbu…', 'truncated': False, 'entities': {'hashtags': [{'text': 'COVID19', 'indices': [34, 42]}, {'text': 'VaccinesWork', 'indices': [66, 79]}], 'symbols': [], 'user_mentions': [{'screen_name': 'redstarneil', 'name': 'Cllr Neil Nerva 🇪🇺🇬🇧🌹🟣🚉', 'id': 65481914, 'id_str': '65481914', 'indices': [3, 15]}, {'screen_name': 'KensalRLibrary', 'name': 'Friends of KRLibrary', 'id': 225438277, 'id_str': '225438277', 'indices': [108, 123]}], 'urls': []}, 'metadata': {'iso_language_code': 'en', 'result_type': 'recent'}, 'source': '<a href="https://mobile.twitter.com" rel="nofollow">Twitter Web App</a>', 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_us

Chaque Tweet comporte de nombreux attributs qui décrivent le contexte dans lequel le tweet a été publié (ex : géolocalisation du tweet, profil de l’auteur du tweet, nombre de followers, etc..). 

Pour ce projet, nous avons sélectionné les attributs suivants :

- **Tweet_id** : Identifiant du tweet 
- **User_name** : le nom de l’utilisateur : Auteur du tweet 
- **Date** : Date de création et l’heure.
- **Text** : Message du tweet (contenu textuel)
- **User_id** : Identifiant de l’utilisateur
- **User_statuses_count** : Le nombre de Tweets (y compris les retweets) émis par l'utilisateur.
- **User_followers** : Le nombre d'abonnés que le compte a actuellement.
- **User_location** : L'emplacement défini par l'utilisateur pour le profil de ce compte. 
- **Like_count** : Le nombre de Tweets que cet utilisateur a emis pendant la durée de vie du compte.
- **Retweet_count** : Indicateur que ce tweet est un tweet Retweeté, et le nombre de retweet.
- **User_verified** : Si True, indique que l'utilisateur a un compte vérifié.


La commande **DataFrame** permet d’organiser toutes nos données sur les tweets dans un tableau pandas. On exporte ensuite le dataframe au format **csv** sous le nom : « **tweet_extraction_covid19_vaccine** ».

In [8]:
df = pd.DataFrame(columns = ['tweet_id', 'user_name', 'date', 'text', 'user_id', 'user_statuses_count', 'user_followers', 'user_location', 'like_count', 'retweet_count', 'user_verified'])

In [9]:
def stream(data, file_name):
    i = 0
    for tweet in tweepy.Cursor(api.search, q=data, count=100, lang='en').items():
        print(i,end='\r')
        df.loc[i, 'tweet_id'] = tweet.id
        df.loc[i, 'user_name'] = tweet.user.name
        df.loc[i, 'date'] = tweet.created_at
        df.loc[i, 'text'] = tweet.text
        df.loc[i, 'user_id'] = tweet.user.id
        df.loc[i, 'user_statuses_count'] = tweet.user.statuses_count
        df.loc[i, 'user_followers'] = tweet.user.followers_count
        df.loc[i, 'user_location'] = tweet.user.location
        df.loc[i, 'like_count'] = tweet.favorite_count
        df.loc[i, 'retweet_count'] = tweet.retweet_count
        df.loc[i, 'user_verified'] = tweet.user.verified
        df.to_csv('{}.csv'.format(file_name))
        i+=1
        if i == 10000:
            break
        else:
            pass
        

In [10]:
stream(data = ['covid19 vaccine'], file_name = 'tweet_extraction_covid19_vaccine')

Tous les tweets ont été créés le **11/01/2022**, ce qui représente un total de **9999** tweets à analyser.

In [11]:
df.head()

,tweet_id,user_name,date,text,user_id,user_statuses_count,user_followers,user_location,like_count,retweet_count,user_verified
0,1480995707943211008,Colorado Department of Public Health & Environ...,2022-01-11 20:11:07,"📍Chapel Hills Mall\n1710 Briargate Blvd, Color...",331244103,9176,50810,Colorado,0,0,True
1,1480995703216164864,Colorado Department of Public Health & Environ...,2022-01-11 20:11:05,"📍Ball Arena\n1000 Chopper Cir, Denver, CO 8020...",331244103,9176,50810,Colorado,0,0,True
2,1480995693279862784,Colorado Department of Public Health & Environ...,2022-01-11 20:11:03,📍Arapahoe Community College (Lot B)\n5900 S Sa...,331244103,9176,50810,Colorado,0,0,True
3,1480995600946458624,Savannah Deschner,2022-01-11 20:10:41,"RT @SKGov: #COVID19SK Summary for January 11, ...",886614449423036417,15562,247,Regina SK Canada,0,6,False
4,1480995572937015299,David Mossey,2022-01-11 20:10:34,RT @CityNewsTO: A new policy mandating #COVID1...,2392851672,30042,439,,0,5,False
